In [1]:
from utils import extract_scaling_metadata
from pathlib import Path
import czifile
import napari
import numpy as np
import skimage
from skimage import measure, exposure
from skimage.measure import regionprops_table
import pandas as pd
import pyclesperanto_prototype as cle
import matplotlib.pyplot as plt

In [8]:
directory_path = Path("./raw_data/")
images = []

# Iterate through the lsm files in the directory
for file_path in directory_path.glob("*.czi"):
    # Remove Control and Isotype stainings from the analysis
    if "Control" and "Isotype" not in file_path.stem:
        images.append(str(file_path))
    
len(images)

117

In [12]:
image = images[0]
ids = []

for image in images:

    # Extract filename and well_id
    file_path = Path(image)
    filename = file_path.stem

    # Extract experiment, mouse, treatment and replica ids
    experiment_id = filename.split(" ")[0]
    mouse_id = filename.split(" ")[1]
    treatment_id = filename.split(" ")[2]
    replica_id = filename.split(" ")[-1]

    # Create a dictionary and transform into a Dataframe to check file names

    identifier = {
        "experiment_id":experiment_id,
        "mouse_id":mouse_id,
        "treatment_id":treatment_id,
        "replica_id":replica_id
    }

    ids.append(identifier)

df_ids = pd.DataFrame(ids)

df.to_csv("./ids.csv")

In [29]:
import napari
viewer = napari.Viewer(ndisplay=2)

image = images[0]
file_path = Path(image)
filename = file_path.stem

# Read the image file and remove singleton dimensions
img = czifile.imread(image)
img = img.squeeze()

# Extract experiment, mouse, treatment and replica ids
experiment_id = filename.split(" ")[0]
mouse_id = filename.split(" ")[1]
treatment_id = filename.split(" ")[2]
replica_id = filename.split(" ")[-1]

# Image size reduction to improve processing times (slicing, not lossless compression)
slicing_factor = None # Use 2 or 4 for compression (None for lossless)

# Extract the stack containing the UEA-1 (0), YAP (1), nuclei (2) and BCAT (3) channels.
uea1_stack = img[0, :, ::slicing_factor, ::slicing_factor]
yap_stack = img[1, :, ::slicing_factor, ::slicing_factor]
nuclei_stack = img[2, :, ::slicing_factor, ::slicing_factor]
bcat_stack = img[3, :, ::slicing_factor, ::slicing_factor]

# Extract x,y,z scaling from .czi file metadata in order to make data isotropic
scaling_x_um, scaling_y_um, scaling_z_um = extract_scaling_metadata(file_path)

# Resample for Voronoi-Otsu labeling
resampled_nuclei = cle.scale(nuclei_stack, factor_x=scaling_x_um, factor_y=scaling_y_um, factor_z=scaling_z_um, auto_size=True)
background_subtracted = cle.top_hat_box(resampled_nuclei, radius_x=5, radius_y=5, radius_z=5)
segmented = cle.voronoi_otsu_labeling(background_subtracted, spot_sigma=2, outline_sigma=2)
del background_subtracted

resampled_yap = cle.scale(yap_stack, factor_x=scaling_x_um, factor_y=scaling_y_um, factor_z=scaling_z_um, auto_size=True)
        
nuclei_labels_3d = cle.pull(segmented)
resampled_yap = cle.pull(resampled_yap)


viewer.add_image(resampled_nuclei)
viewer.add_labels(nuclei_labels_3d)

# Extract regionprops
props = regionprops_table(label_image=nuclei_labels_3d, intensity_image=resampled_yap, properties=["label", "intensity_mean", "centroid", "area_filled"])

# Construct a dataframe for the extracted props
df = pd.DataFrame(props)



In [21]:
resampled = cle.scale(img[2,:,:,:], factor_x=scaling_x_um, factor_y=scaling_y_um, factor_z=scaling_z_um, auto_size=True)

In [18]:
print(scaling_x_um, scaling_y_um, scaling_z_um)

0.42 0.42 1.0


In [24]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(cle.pull(resampled))

<Image layer 'Image' at 0x1fc947b97c0>